In [1]:
import matplotlib.pyplot as plt
import chess.pgn
import numpy as np 
import chess

pgn = open("D:\\projets\\chess\\data\\lichess_elite_2020-06.pgn")

In [2]:
def get_base_vect(size):
    arr = [0 for i in range(size)]
    return np.array(arr)

def get_vect(board):
    pieces = [chess.PAWN, chess.KNIGHT, chess.BISHOP, chess.ROOK, chess.QUEEN, chess.KING]
    colors = [chess.WHITE, chess.BLACK]
    vect = get_base_vect(len(pieces)*2*64)
    for i in range(len(colors)):
        for e in range(len(pieces)):
            lst = board.pieces(color=colors[i], piece_type=pieces[e])
            for position in lst:
                vect[position*len(pieces)*2 + i*len(pieces) + e] = 1
    return vect

In [20]:
X, Y = [], []
Y_ = []

count = 0

for i in range(10000):
   game = chess.pgn.read_game(pgn)
   if game.headers["Result"] == "1/2-1/2":
      continue
   winner = 0
   y_ = 0
   if game.headers["Result"] == "1-0":
      y_ = 1 
   if game.headers["Result"] == "0-1":
      y_ = 2

   vectY = np.zeros(3)
   vectY[y_] = 1
   board = game.board()

   for move in game.mainline_moves():
      board.push(move)
      count += 1
      if count%5 == 0:
         vect = get_vect(board)
         X.append(vect)
         Y.append(y_)
         Y_.append(vectY)
   
   if i%1000 == 0:
      print(i, len(X))

print(len(X))

144398


In [21]:
X = np.array(X)
Y = np.array(Y)
Y1 = np.array(Y_)

print(X.shape)
print(Y.shape)

(144398, 768)
(144398,)


In [24]:
import tensorflow.keras as keras

model = keras.Sequential()
#  kernel_regularizer=keras.regularizers.L1(0.001)
model.add(keras.layers.Dense(128, activation="relu", kernel_regularizer=keras.regularizers.L1(0.001), input_shape=(len(X[0]),)))
model.add(keras.layers.Dense(64, kernel_regularizer=keras.regularizers.L1(0.0001), activation="relu"))
model.add(keras.layers.Dense(3, activation="softmax"))

opt = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=["accuracy"])

model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_13 (Dense)            (None, 128)               98432     
                                                                 
 dense_14 (Dense)            (None, 64)                8256      
                                                                 
 dense_15 (Dense)            (None, 3)                 195       
                                                                 
Total params: 106,883
Trainable params: 106,883
Non-trainable params: 0
_________________________________________________________________


In [25]:
split = int(len(X)*0.75)

from sklearn.utils import shuffle
X_, Y_ = shuffle(X[0:split], Y1[0:split], random_state=0)

hist = model.fit(X_, Y_, batch_size=128, epochs=50, validation_data=(X[split:len(X)], Y1[split:len(Y)]),)

plt.plot(hist.history["loss"])
plt.plot(hist.history["val_loss"])
plt.show()

plt.plot(hist.history["accuracy"])
plt.plot(hist.history["val_accuracy"])
plt.show()

Epoch 1/50
847/847 [==============================] - 4s 4ms/step - loss: 2.3581 - accuracy: 0.5242 - val_loss: 0.9577 - val_accuracy: 0.5338
Epoch 2/50
847/847 [==============================] - 4s 5ms/step - loss: 0.8514 - accuracy: 0.5473 - val_loss: 0.7998 - val_accuracy: 0.5344
Epoch 3/50
847/847 [==============================] - 4s 4ms/step - loss: 0.7736 - accuracy: 0.5448 - val_loss: 0.7594 - val_accuracy: 0.5394
Epoch 4/50
847/847 [==============================] - 4s 5ms/step - loss: 0.7465 - accuracy: 0.5429 - val_loss: 0.7396 - val_accuracy: 0.5390
Epoch 5/50
847/847 [==============================] - 4s 5ms/step - loss: 0.7325 - accuracy: 0.5420 - val_loss: 0.7287 - val_accuracy: 0.5381
Epoch 6/50
847/847 [==============================] - 3s 4ms/step - loss: 0.7242 - accuracy: 0.5424 - val_loss: 0.7221 - val_accuracy: 0.5344
Epoch 7/50
847/847 [==============================] - 3s 4ms/step - loss: 0.7186 - accuracy: 0.5440 - val_loss: 0.7173 - val_accuracy: 0.5406
Epoch 

KeyboardInterrupt: 